<a href="https://colab.research.google.com/github/joalmo/JAlonMor/blob/main/TFM_WebScrapV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalo fake_useragent

! pip install fake_useragent
! pip install xmltojson
! pip install utils

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=3518fe4bade347afe7484244bad2dc1fa76ab0629e293f4110fb34e7395182b9
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [ ]:
import requests
import json
import csv
import pandas as pd
import numpy as np
from IPython.core.display import HTML
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib import request as urllib2
import re
from fake_useragent import UserAgent
import random

import time

import http.client  # or http.client if you're on Python 3
http.client._MAXHEADERS = 1000

Al intentar scrapear la web de coches.net, la página detecta que soy un bot, y no me transmite información. Para evitar este control, utilizo unos headers determinados y uso la librería UserAgent, para que cada conexión utilice un agente aleatorio distinto.

In [ ]:
url_base = 'https://www.coches.net/segunda-mano/?MakeId=28&MinYear=2014&ModelId=8'


# https://stackoverflow.com/questions/52068920/python-requests-beautifulsoup-bot-detection

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
ua=UserAgent()
hdr = {'User-Agent': ua.random,
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,es;q=0.9,en;q=0.8',
      'Connection': 'keep-alive'}

r = requests.get(url_base, headers=hdr)


# Extraer HTML
html = r.content

# Convertir a BeatifulSoup
soup  = BeautifulSoup(html,'lxml')


In [ ]:
r.status_code

200

Ahora he obtenido un HTML, pero no corresponde con la realizadad de la página que podemos ver en el navegador. Parece que utiliza JavaScript para generar el HTML, indicando en el código las claves-valores necesarias mediante JSON. Voy buscar la información relevante en este script para poder descargar los datos necesarios.

In [ ]:
scripts = soup.find_all('script')
scripts

[<script>!function(win){
       function params(){var a=win.location.search.substring(1).split('&'),obj={},b,i;for(i in a){if(a[i]==='')continue;b=a[i].split('=');obj[decodeURIComponent(b[0])]=decodeURIComponent(b[1])}return obj;}if(params().thirdparties==='false'){win.loadThirdParties=false;return};win.loadThirdParties=true;
       var keys,siteKeys={coches:{production:"RYnPTD8KL55ybR0yBc24Mgz2BVkquF2g",default:"v4TfGwIp3OSaH8U7TJBO2cdsHFGaPrUA"},motos:{production:"TX2xa4CL8eNRdVC7Yz43On3sNesy595g",default:"jP3sEVXcDVeTx0U0jg5Hk7BwMLmE82MA"}}["coches"],envKey,writeKey=siteKeys["production"]||siteKeys.default,analytics=win.analytics=win.analytics||[];if(!analytics.initialize)if(analytics.invoked)win.console&&console.error&&console.error("Segment snippet included twice.");else{analytics.invoked=!0;analytics.methods=["trackSubmit","trackClick","trackLink","trackForm","pageview","identify","reset","group","track","ready","alias","debug","page","once","off","on","addSourceMiddleware","addI

Podemos ver que el script que me interesa es el que se encuentra en penúltima posición. Observando la estructura del HTML, probablemente el patrón se repetirá en las siguientes páginas de la web de coches.net. 
Selecciono el penúltimo elemento

In [ ]:
script_coches = scripts[-2]
script_coches

<script>window.__INITIAL_PROPS__ = JSON.parse("{\"breadcrumb\":[{\"label\":\"Coches de segunda mano\",\"url\":\"https://www.coches.net/segunda-mano/\"},{\"label\":\"MERCEDES-BENZ\",\"url\":\"https://www.coches.net/mercedes-benz/segunda-mano/\"},{\"label\":\"Clase C\",\"url\":\"https://www.coches.net/mercedes-benz/clase_c/segunda-mano/\"},{\"label\":\"COMMON_SEARCH_ENGINE\",\"params\":{\"siteName\":\"coches\"}}],\"h1\":\"MERCEDES-BENZ Clase C de segunda mano y ocasi\u00F3n\",\"helmet\":{\"title\":\"MERCEDES-BENZ Clase C de segunda mano y ocasi\u00F3n | Coches.net\",\"meta\":[{\"name\":\"description\",\"content\":\"1997 MERCEDES-BENZ Clase C de particulares y concesionarios de segunda mano y ocasi\u00F3n. \u00A1Encuentra MERCEDES-BENZ Clase C al mejor precio!\"},{\"name\":\"robots\",\"content\":\"INDEX, FOLLOW\"}],\"link\":[{\"rel\":\"canonical\",\"href\":\"https://www.coches.net/mercedes-benz/clase_c/segunda-mano/\"},{\"rel\":\"preload\",\"as\":\"image\",\"src\":\"https://a.ccdn.es/cnet

Extraigo la información y lo exporto a archivo JSON para su posterior análisis. 

In [ ]:
# mil gracias a este señor!! https://morioh.com/p/4ed4ba1fe438

js = 'window = {};\n'+script_coches.text.strip()+';\nprocess.stdout.write(JSON.stringify(window.__INITIAL_PROPS__));'

with open('temp.js','w') as f:
    f.write(js)

from subprocess import check_output
script_JS = check_output(['node','temp.js'])
print(json.loads(script_JS))

{'breadcrumb': [{'label': 'Coches de segunda mano', 'url': 'https://www.coches.net/segunda-mano/'}, {'label': 'MERCEDES-BENZ', 'url': 'https://www.coches.net/mercedes-benz/segunda-mano/'}, {'label': 'Clase C', 'url': 'https://www.coches.net/mercedes-benz/clase_c/segunda-mano/'}, {'label': 'COMMON_SEARCH_ENGINE', 'params': {'siteName': 'coches'}}], 'h1': 'MERCEDES-BENZ Clase C de segunda mano y ocasión', 'helmet': {'title': 'MERCEDES-BENZ Clase C de segunda mano y ocasión | Coches.net', 'meta': [{'name': 'description', 'content': '1997 MERCEDES-BENZ Clase C de particulares y concesionarios de segunda mano y ocasión. ¡Encuentra MERCEDES-BENZ Clase C al mejor precio!'}, {'name': 'robots', 'content': 'INDEX, FOLLOW'}], 'link': [{'rel': 'canonical', 'href': 'https://www.coches.net/mercedes-benz/clase_c/segunda-mano/'}, {'rel': 'preload', 'as': 'image', 'src': 'https://a.ccdn.es/cnet/2021/11/05/50477610/511735148_g.jpg/400x273cut/'}], 'scripts': [{'type': 'application/ld+json', 'innerHTML': 

Creo objeto JSON y estudio su información

In [ ]:
info_coches_js = json.loads(script_JS)
info_coches_js.keys()

dict_keys(['breadcrumb', 'h1', 'helmet', 'structuredDataJsonList', 'initialSearch', 'listFiltersOptions', 'initialResults', 'pageName', 'pageErrors', 'paginationPlaceholderUrl', 'notifications', 'hasComeFromAdNotAvailable', 'tagclouds', 'trackingData'])

In [ ]:
info_coches_js['initialResults']

{'items': [{'bodyTypeId': 4,
   'financedPrice': 28700,
   'fuelTypeId': 1,
   'hasWarranty': True,
   'id': '50477610',
   'img': 'https://a.ccdn.es/cnet/2021/11/05/50477610/511735148_g.jpg',
   'imgUrl': 'https://a.ccdn.es/cnet/2021/11/05/50477610/511735148_g.jpg/359x269cut/',
   'includesTaxes': True,
   'isCertified': False,
   'isFinanced': True,
   'isProfessional': True,
   'isTop': True,
   'isUrgent': False,
   'isUrlSemantic': True,
   'km': 62270,
   'location': {'mainProvince': 'La Rioja',
    'mainProvinceId': 26,
    'provinceIds': [26]},
   'offerType': {'id': 0, 'literal': 'Ocasión'},
   'phone': '941621900',
   'photos': ['https://a.ccdn.es/cnet/2021/11/05/50477610/511735148_g.jpg',
    'https://a.ccdn.es/cnet/2021/11/05/50477610/511735150_g.jpg',
    'https://a.ccdn.es/cnet/2021/11/05/50477610/511735151_g.jpg',
    'https://a.ccdn.es/cnet/2021/11/05/50477610/511735153_g.jpg',
    'https://a.ccdn.es/cnet/2021/11/05/50477610/511735155_g.jpg',
    'https://a.ccdn.es/cnet

In [ ]:
info_coches_js['initialResults']['items'][25]

{'bodyTypeId': 4,
 'financedPrice': 17890,
 'fuelTypeId': 1,
 'hasWarranty': True,
 'id': '51752724',
 'img': 'https://a.ccdn.es/cnet/2022/04/08/51752724/559467054_g.jpg',
 'imgUrl': 'https://a.ccdn.es/cnet/2022/04/08/51752724/559467054_g.jpg/359x269cut/',
 'includesTaxes': True,
 'isCertified': False,
 'isFinanced': True,
 'isProfessional': True,
 'isUrgent': False,
 'isUrlSemantic': True,
 'km': 130324,
 'location': {'mainProvince': 'Madrid',
  'mainProvinceId': 28,
  'provinceIds': [28, 36]},
 'offerType': {'id': 0, 'literal': 'Ocasión'},
 'phone': '910602302',
 'photos': ['https://a.ccdn.es/cnet/2022/04/08/51752724/559467054_g.jpg',
  'https://a.ccdn.es/cnet/2022/04/08/51752724/559467064_g.jpg',
  'https://a.ccdn.es/cnet/2022/04/08/51752724/559467076_g.jpg',
  'https://a.ccdn.es/cnet/2022/04/08/51752724/559467083_g.jpg',
  'https://a.ccdn.es/cnet/2022/04/08/51752724/559467098_g.jpg',
  'https://a.ccdn.es/cnet/2022/04/08/51752724/559467105_g.jpg',
  'https://a.ccdn.es/cnet/2022/04/0

In [ ]:
print(len(info_coches_js['initialResults']['items']))

35


In [ ]:
info_coches_js['initialResults']['items'][25]['location']['mainProvince']

'Madrid'

In [ ]:
info_coches_js['initialResults']['items'][4]['price']

27950

In [ ]:
info_coches_js['initialResults']['items'][25]['title']

'MERCEDES-BENZ Clase C C 220 d Estate 5p.'

In [ ]:
info_coches_js['initialResults']['items'][34]['km']

83900

In [ ]:
info_coches_js['initialResults']['items'][4]['fuelTypeId']

1

In [ ]:
info_coches_js['initialResults']['items'][4]['year']

2017

In [ ]:
info_coches_js['initialResults']['items'][4]['url']

'/mercedes-benz-clase-c-c-220-d-4p-diesel-2017-en-valencia-51672450-covo.aspx'

In [ ]:
info_coches_js['initialResults']['items'][4]['hasWarranty']

True

In [ ]:
import random

In [ ]:
html = []
pagina = 0
status = 200

# Funciones

def check_url(pagina):
    
    languaje = ['fr-CH, fr', 'en-US,es', 'es-ES,es', 'en-US,en', 'de-CH', 'es']
    languaje_header = random.choice(languaje)
    
    url_base = 'https://www.coches.net/segunda-mano/?MakeId=28&ModelId=8&MinYear=2014&pg=' +str(pagina)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    ua=UserAgent()
    hdr = {'User-Agent': ua.random,
          'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': languaje_header, #+ ';q=0.9,en;q=0.8'
          'Connection': 'keep-alive'}

    r = requests.get(url_base, headers=hdr)
    html = r.content
    status = r.status_code
    return status, html


# Repito el proceso automatizado #

In [ ]:
# Defino archivo de salida
f_csv = '/content/drive/MyDrive/Colab Notebooks/TFM/coches_net.csv'#'coches_net.csv'


with open(f_csv, 'w') as f:

  writer = csv.writer(f)

  writer.writerow(['titulo', 'precio', 'kilometros', 'año', 'combustible', 'ciudad', 'garantia', 'url'])

  # Contador segun status html
  status = 0
  pagina = 0
  es_ultima = False

  # Genero una búsqueda en las primeras 65 páginas, con lo obtendré más de 2000 resultados
  
  while es_ultima == False:

    pagina = pagina +1
    
    time.sleep(random.randrange(7,22,1))
    url_result = check_url(pagina)
    html_url = url_result[1]
    status = len(html_url)

    if status >= 10000:
      
      print('pagina ', pagina)
        
    
      # Convertir a BeatifulSoup
      soup  = BeautifulSoup(html_url,'lxml')

      # Compruebo si es la última página
      paginacion = soup.find_all('li', attrs={'class':'sui-MoleculePagination-item'})
      paginacion
      pagina_siguiente = paginacion[-1].find('a').get('href')

      if 'null' in pagina_siguiente:
        es_ultima == True
      else:
        es_ultima == False


      scripts = soup.find_all('script')
      script_coches = scripts[-2]

      #print(scripts)

      # Extraigo script
      js = 'window = {};\n'+script_coches.text.strip()+';\nprocess.stdout.write(JSON.stringify(window.__INITIAL_PROPS__));'

      # Exporto a archivo temporal .js y creo objeto JSON
      with open('temp.js','w') as f:
        f.write(js)

      from subprocess import check_output
      script_JS = check_output(['node','temp.js'])

      info_coches_js = json.loads(script_JS)

      # cálculo número de anuncios en cada pagina
      items = len(info_coches_js['initialResults']['items'])
    
      # Itero por todos los elementos de interés del JSON
      for item in range(0,items-1):

        titulo = info_coches_js['initialResults']['items'][item]['title'] if 'title' in info_coches_js['initialResults']['items'][item] else None
        precio = info_coches_js['initialResults']['items'][item]['price'] if 'price' in info_coches_js['initialResults']['items'][item] else None
        kilometros = info_coches_js['initialResults']['items'][item]['km'] if 'km' in info_coches_js['initialResults']['items'][item] else None
        año = info_coches_js['initialResults']['items'][item]['year'] if 'year' in info_coches_js['initialResults']['items'][item] else None
        combustible = info_coches_js['initialResults']['items'][item]['fuelTypeId'] if 'fuelTypeId' in info_coches_js['initialResults']['items'][item] else None
        ciudad = info_coches_js['initialResults']['items'][item]['location']['mainProvince'] if 'mainProvince' in info_coches_js['initialResults']['items'][item]['location'] else None
        garantia = info_coches_js['initialResults']['items'][item]['hasWarranty'] if 'hasWarranty' in info_coches_js['initialResults']['items'][item] else None
        url = info_coches_js['initialResults']['items'][item]['url'] if 'url' in info_coches_js['initialResults']['items'][item] else None

        '''
        titulo = info_coches_js['initialResults']['items'][item]['title']
        precio = info_coches_js['initialResults']['items'][item]['price']
        kilometros = info_coches_js['initialResults']['items'][item]['km']
        año = info_coches_js['initialResults']['items'][item]['year']
        combustible = info_coches_js['initialResults']['items'][item]['fuelTypeId']
        ciudad = info_coches_js['initialResults']['items'][item]['location']['mainProvince']
        garantia = info_coches_js['initialResults']['items'][item]['hasWarranty']
        url = info_coches_js['initialResults']['items'][item]['url']
        '''

        writer.writerow([titulo, precio, kilometros, año, combustible, ciudad, garantia, url])



    else:

      time.sleep(30 + random.randrange(17,200,1))
      pagina = pagina - 1
      es_ultima == False
      
    
    
print(status)
print('FIN')











Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


pagina  1
pagina  2
pagina  3
pagina  4
pagina  5
pagina  6
pagina  7
pagina  8
pagina  9
pagina  10
pagina  11
pagina  12
pagina  13
pagina  14
pagina  15
pagina  16
pagina  17
pagina  18
pagina  19
pagina  20
pagina  21
pagina  22
pagina  23
pagina  24
pagina  25
pagina  26
pagina  27
pagina  28
pagina  29
pagina  30
pagina  31
pagina  32
pagina  33
pagina  34
pagina  35
pagina  36
pagina  37
pagina  38
pagina  39
pagina  40
pagina  41
pagina  42
pagina  43
pagina  44
pagina  45
pagina  46
pagina  47
pagina  48
pagina  49
pagina  50
pagina  51
pagina  52
pagina  53
pagina  54
pagina  55
pagina  56
pagina  57
pagina  58
pagina  59
pagina  60
pagina  61
pagina  62
pagina  63
pagina  64
pagina  65
pagina  66
pagina  67
pagina  68


IndexError: ignored

Añadir condición cuando parseo JSON, de si al´gun campo no tiene valor, poner Null. None

In [ ]:

x = x+2y

None


In [ ]:
# que lento

0 1
1 1
2 1
3 1
4 2
5 2
6 1
7 1
8 1
9 1
10 4
11 1
12 1
13 1
14 1
15 4
16 1
17 1
18 1
19 2


KeyError: ignored

In [ ]:
for item in range(21,items-1):


  print(item, info_coches_js['initialResults']['items'][item]['fuelTypeId'])

21 1
22 1
23 1
24 1
25 4
26 1
27 1
28 1


In [ ]:
a = None

In [ ]:
print(a)

None


In [ ]:
! python -V

Python 3.7.13


Seguir como en este ejemplo, más o menos...

https://wiki.montera34.com/taller-web-scraping-hirikilabs/coches



In [ ]:
!

sample_data


In [ ]:
# https://stackoverflow.com/questions/35120250/python-3-get-and-parse-json-api
# https://es.stackoverflow.com/questions/133147/attributeerror-nonetype-object-has-no-attribute-group
# https://stackoverflow.com/questions/48821725/xml-parsers-expat-expaterror-not-well-formed-invalid-token
# https://stackoverflow.com/questions/63666155/how-can-i-get-this-json-value-using-beautifulsoup-from-window-initial-state

In [ ]:
! cd..

/bin/bash: cd..: command not found


In [ ]:
! cd ..

In [ ]:
!ls

sample_data


In [ ]:
! cd bin/

/bin/bash: line 0: cd: bin/: No such file or directory
